In [1]:
#对微博新闻句子进行检索
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [2]:
# 读取文件
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号替换成空字符串,将所有的句子放到一个list种方便遍历
sentences = re.split('[。！？]', text.replace('\n', ''))


In [3]:
# 最后一行如果为空，则删除,目的为去掉空行
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [4]:
# 利用jieba将item_text进行分词,每个单词用空格作为间隔，将输入的文本分词，并转换成list类型并返回
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):        
            item_str += i.word
            item_str += " "
    return item_str


In [5]:
# 第一步进行分词
documents = []
for item_text in sentences:
    # 将item_text进行分词，方法将分词后的list入到数组中。
    item_str = get_item_str(item_text)
    documents.append(item_str)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Dracula\AppData\Local\Temp\jieba.cache
Loading model cost 0.611 seconds.
Prefix dict has been built successfully.


In [6]:
# 第二步，创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp


In [7]:
# 第三步，创建LSH Forest及MinHash对象
minhash_list = []
  #创建LSH Forest
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)


In [8]:
# 创建索引，之后可用索引查找
forest.index()


In [9]:
query = '里皮辞职返回意大利，他的助教马达洛尼随队返回广州。'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)


In [10]:
# 第四步，查询forest相似的句子top3
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)


24 0.125 马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事
25 0.1640625 ”中国足协：接受里皮辞职请求，将深刻反思看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定
23 0.921875 ”里皮辞职返回意大利，他的助教马达洛尼随队返回广州
Top 3 邻居 [24, 25, 23]


In [11]:
# 第五步，查询top10对比forest方法查询出来的结果
result = forest.query(minhash_query, 10)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)


0 0.171875 #斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用
2 0.171875 谁将成为新主帅，成为广大球迷关注的焦点
3 0.125 目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里
11 0.0859375 来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足
12 0.234375 老头有点意思，凡是里皮干过的地方，他就想试试
17 0.1171875 中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有
20 0.1015625 主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队
23 0.921875 ”里皮辞职返回意大利，他的助教马达洛尼随队返回广州
24 0.125 马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事
25 0.1640625 ”中国足协：接受里皮辞职请求，将深刻反思看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定
Top 3 邻居 [0, 2, 3, 11, 12, 17, 20, 23, 24, 25]


In [12]:
#根据算出来的jaccard相似度由大到小排列的顺序，按理说top应为 23 25 12    说明ANN方式确实存在误差